In [1]:
import torch
import torchvision
from torch import optim 
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms,datasets

In [2]:
def predict_images(path):
    file = open(path,'r')
    directory = file.read()
    test_img_transforms = transforms.Compose([transforms.Resize(64),
                                          transforms.CenterCrop(64),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                               [0.229, 0.224, 0.225])])

    # Pass transforms in here, then run the next cell to see how the transforms look
    #train_data = datasets.ImageFolder('/home/nilesh/Desktop/MY FILES/hackathon-dataset/raw/apple_data/training', transform=train_transforms)
    test_data = datasets.ImageFolder(directory, transform=test_img_transforms)

    testloader = torch.utils.data.DataLoader(test_data, batch_size=1 , shuffle = True)
    return testloader

In [3]:
class neural_net(nn.Module):
    def __init__(self):
        super().__init__()
        #64*64*3
        self.conv1 = nn.Conv2d(3 , 32 , 3 , padding = 1)
        #32*32*32
        self.conv2 = nn.Conv2d(32 , 64 , 3 , padding = 1)
        #16*16*64
        self.conv3 = nn.Conv2d(64 , 32 , 3 , padding = 1)
        #8*8*32
        self.dropout = nn.Dropout(0.2)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(8*8*32 , 100)
        self.fc2 = nn.Linear(100 , 64)
        self.fc3 = nn.Linear(64 , 32)
        self.fc4 = nn.Linear(32 , 4)
        
        
    def forward(self , x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1 , 8*8*32)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x) , dim = 1)
        return x

In [4]:
import pickle
filename = 'finalized_model1.sav'
loaded_model = pickle.load(open(filename ,'rb'))

In [5]:
loaded_model

neural_net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout): Dropout(p=0.2)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=4, bias=True)
)

In [7]:
def exp_val(path):
    _ , (images , _) = next(enumerate(predict_images(path)))
    file = open('prediction.txt','w+')
    file.write(torch.exp(loaded_model(images)).detch().numpy()[0])